<a href="https://colab.research.google.com/github/dh-conciani/br-open-ecosystem-degradation/blob/main/01c_buildMosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Tutorial: Processamento de Mosaicos no Google Cloud Storage**

Este notebook foi criado para facilitar a organização e o processamento de arquivos **GeoTIFF** armazenados no **Google Cloud Storage (GCS)**. Ele permite criar **mosaicos de imagens**, organizá-los em pastas e enviá-los para um bucket no GCS, de forma **automatizada e interativa**.

---

## ✅ **O que este notebook faz?**
- 📂 **Lista** os arquivos `.tif` disponíveis em um bucket no GCS.
- 🔎 **Verifica** se os mosaicos já foram criados anteriormente.
- 🖼️ **Gera mosaicos** automaticamente para arquivos que precisam ser combinados.
- 🚀 **Envia os mosaicos** finalizados para a pasta correta no GCS.
- 🖱️ **Permite interação** por meio de uma interface onde você pode escolher quais arquivos deseja processar.

---

## 🎯 **O que você precisa antes de começar?**
### 1️⃣ **Configurar o Google Cloud**
- Criar um **bucket** no Google Cloud Storage.
- Garantir que sua conta tem **permissões de leitura e escrita** no bucket.

### 2️⃣ **Instalar as Dependências**
Se necessário, instale as bibliotecas executando o seguinte comando:
```python
!pip install google-cloud-storage gdal tqdm ipywidgets
```

### 3️⃣ **Organizar seus arquivos**
- Seus arquivos `.tif` devem estar armazenados em uma pasta no bucket.
- Os mosaicos gerados serão salvos automaticamente em uma **subpasta organizada por tipo de dado**.

---

## 📜 **Passo a passo: Como usar este notebook**

### 🔹 **1. Configurar as variáveis principais**
Antes de rodar qualquer célula, configure o nome do bucket e os caminhos das pastas:

```python
bucket_name = 'seu-bucket-no-gcs'
input_folder = 'caminho/para/pasta/de/entrada/'
output_folder = 'caminho/para/pasta/de/saida/'
```

Isso garante que o notebook saiba **onde buscar os arquivos e onde salvar os mosaicos**.

---

### 🔹 **2. Carregar os dados**
Agora, execute a célula que carrega as informações dos arquivos disponíveis no GCS. Isso verificará:
✅ Quais arquivos `.tif` estão no bucket.  
✅ Se há arquivos que precisam ser mosaicados.  
✅ Se algum mosaico já foi criado anteriormente.

```python
data_info = load_data_info(bucket_name, input_folder, output_folder)
```

Isso criará um **resumo dos arquivos** que estão prontos para processamento.

---

### 🔹 **3. Escolher os arquivos a processar**
Este notebook tem uma **interface interativa** para facilitar a seleção das bandas a serem processadas.

1️⃣ Execute a célula que **exibe a interface**.  
2️⃣ Veja a lista de **bandas disponíveis**.  
3️⃣ Marque os **checkboxes** para selecionar os arquivos que deseja processar.  
4️⃣ Use os botões para **ligar/desligar** seleções rapidamente.  

💡 **Dica:** Arquivos já mosaicados aparecem como ⚠️ e não podem ser processados novamente.

---

### 🔹 **4. Processar os arquivos selecionados**
Depois de escolher os arquivos na interface, rode a célula para iniciar o processamento:

```python
process_mosaics(selected_bands, data_info, bucket_name, input_folder, output_folder)
```

💡 **O que acontece aqui?**
✅ Arquivos individuais são **copiados para a pasta correta**.  
✅ Arquivos que precisam de mosaico são **processados e salvos**.  
✅ Os arquivos processados são **enviados para o GCS**.  
✅ O progresso e o tempo estimado são exibidos no **console**.

---

## 📌 **Outras informações importantes**
### 🚨 **Posso parar o processo no meio?**
Sim! Se você interromper o notebook, ele **não repetirá os mosaicos já processados**.  
Você pode rodá-lo novamente sem problema.

### 🔍 **Como verificar o progresso?**
Mensagens detalhadas serão exibidas no console, mostrando:
- 🚀 **Início do processamento**  
- ⏳ **Tempo estimado restante**  
- 📂 **Arquivos sendo processados**  
- ✅ **Mosaicos concluídos**  
- ❌ **Erros ou arquivos ausentes**  

---

## 🎉 **Pronto para usar!**
Agora você pode rodar este notebook sempre que precisar organizar e processar novos mosaicos no GCS. Se tiver dúvidas, cheque os logs ou revise os passos acima. 🚀🔥

---

Esse tutorial foi feito para tornar o processo **claro e fácil de seguir**, mesmo para quem não tem experiência em programação. Se precisar tirar duvidas, fazer mais ajustes ou personalizações, entre em contato com wallace.silva@ipam.org.br! 😊

In [ ]:
# -*- coding: utf-8 -*-
"""Mosaico_supercartas_bucket.ipynb

Script base revisado para agregação de informações e preparações do mosaico.

"""

'Mosaico_supercartas_bucket.ipynb\n\nScript base revisado para agregação de informações e preparações do mosaico.\n\n'

In [1]:
# Passo 1: Configurar o ambiente
!apt-get update
!apt-get install -y gdal-bin python3-gdal
!pip install gdal
!pip install tqdm
!pip install google-cloud-storage

from google.colab import auth
auth.authenticate_user()

from google.cloud import storage
import os
import subprocess
import time
import re
from tqdm import tqdm

import ipywidgets as widgets
from IPython.display import display, clear_output


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,471 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,574 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages 

In [2]:
# shared-development-storage/COLECOES/BRASIL/COLECAO9/AGRICULTURE
# Configurações do Google Cloud Storage
bucket_name = 'shared-development-storage'

# BUCKET DO GOOGLE CLOUD COM OS DADOS: https://console.cloud.google.com/storage/browser/shared-development-storage/AUXILIARES/SEEG_CIAT/LEGAL_AMAZON_b

input_folder = 'AUXILIARES/DEGRADACAO/COL_10/temp'
output_folder = 'AUXILIARES/DEGRADACAO/COL_10'

client = storage.Client()
bucket = client.bucket(bucket_name)


In [3]:


def list_files_in_folder(bucket_name, folder_path):
    """
    Lista os arquivos .tif em uma pasta e retorna apenas os arquivos válidos.
    """
    files = []
    blobs = client.list_blobs(bucket_name, prefix=folder_path)
    for blob in blobs:
        if blob.name.lower().endswith('.tif'):
            files.append(blob.name)
    return files


def identify_data_types_and_bands(files, output_folder, bucket_name):
    """
    Identifica corretamente os tipos de dados e as bandas, separando arquivos únicos dos que precisam ser mosaicados.

    - O tipo de dado é sempre o primeiro elemento ao fazer split('-').
    - A banda é sempre o segundo elemento ao fazer split('-'), mantendo _YYYY ou _YYYY_YYYY.
    - Se o arquivo for um mosaico, remover os últimos 21 caracteres antes de pegar o nome da banda.
    - Se o arquivo não for um mosaico, apenas remover a extensão .tif.

    Args:
        files (list): Lista de caminhos de arquivos.
        output_folder (str): Pasta onde os mosaicos são armazenados.
        bucket_name (str): Nome do bucket para verificar se o mosaico já existe.

    Returns:
        dict: Estrutura contendo tipos de dados e bandas associadas.
    """
    data_info = {}

    for f in files:
        filename = os.path.basename(f)

        # Garantir que o arquivo é um TIFF
        if not filename.endswith('.tif'):
            print(f"Aviso: O arquivo {filename} não é um TIFF válido. Ignorando...")
            continue

        # Remover extensão
        base_name = filename[:-4]

        # Identificar se o arquivo precisa ser mosaicado pelo padrão numérico no final (21 caracteres)
        is_mosaic = re.search(r"\d{10}-\d{10}$", base_name)

        if is_mosaic:
            clean_name = base_name[:-21]  # Remover os 21 caracteres finais
        else:
            clean_name = base_name  # Arquivo único

        # O tipo de dado é sempre o primeiro elemento do split('-')
        parts = clean_name.split('-')
        data_type = parts[0]  # Primeiro elemento é o tipo de dado

        # O nome da banda é sempre o segundo elemento do split('-')
        if len(parts) > 1:
            band = parts[1]
        else:
            print(f"Aviso: O nome do arquivo {filename} não tem estrutura esperada para a banda. Ignorando...")
            continue

        # Verificar se o mosaico já existe
        try:
            mosaic_exists = check_if_mosaic_exists(bucket_name, output_folder, data_type, band)
        except Exception as e:
            print(f"Erro ao verificar mosaico para {filename}: {e}")
            mosaic_exists = False

        # Organizar os dados no dicionário
        if data_type not in data_info:
            data_info[data_type] = {'bands': {}, 'requires_mosaic': bool(is_mosaic)}

        if band not in data_info[data_type]['bands']:
            data_info[data_type]['bands'][band] = {'files': [], 'mosaic_exists': mosaic_exists}

        # Se for mosaico, adicionar na lista de arquivos para mosaico
        if is_mosaic:
            data_info[data_type]['bands'][band]['files'].append(f)
        else:
            # Se não for mosaico, apenas armazenar como um único arquivo
            data_info[data_type]['bands'][band]['file'] = f

    # Debugging para verificar se os dados estão corretos antes de enviar para a interface
    print("\n🔍 Tipos de dados encontrados:", list(data_info.keys()))
    for dt, info in data_info.items():
        print(f"\n🗂️ Tipo de dado: {dt}")
        print("🎵 Bandas:", list(info["bands"].keys()))

    return data_info


def check_if_mosaic_exists(bucket_name, output_folder, data_type, band):
    """
    Verifica se um arquivo mosaico já existe no bucket na subpasta correspondente.
    """
    mosaic_path = f"{output_folder.rstrip('/')}/{data_type}/{data_type}-{band}.tif"
    blob = bucket.blob(mosaic_path)
    return blob.exists()


def mosaic_images(input_files, output_file):
    if not input_files:
        print("[ERRO] Nenhum arquivo de entrada fornecido para a função mosaic_images.")
        return False

    # Escrever a lista de arquivos em um arquivo temporário
    with open('input_files.txt', 'w') as f:
        for file_path in input_files:
            f.write(f"{file_path}\n")

    # Criar arquivo VRT temporário
    vrt_file = 'temp.vrt'
    build_vrt_command = f"gdalbuildvrt -input_file_list input_files.txt {vrt_file}"
    res = os.system(build_vrt_command)
    if res != 0:
        print("[ERRO] gdalbuildvrt falhou ao criar o VRT. Código de saída:", res)
        # Remover arquivos temporários
        if os.path.exists('input_files.txt'):
            os.remove('input_files.txt')
        return False

    # Otimizações no GDAL para salvar imagens mais leves
    translate_command = [
        "gdal_translate",                          # Comando principal do GDAL para conversão de formato
        "-of", "GTiff",                            # Define o formato de saída como GeoTIFF
        "-ot", "Byte",                             # Tipo de dado de saída: 8 bits sem sinal (0-255). Use "Int16" se necessário
        "-co", "TILED=YES",                        # Cria a saída em blocos (tiled), melhorando performance de leitura
        "-co", "COMPRESS=LZW",                    # Usa compressão ZSTD para melhor eficiência (alternativa a DEFLATE ou LZW)
        "-co", "PREDICTOR=2",                      # Ajusta o preditor para dados inteiros (melhor compressão)
        "-co", "ZLEVEL=9",                         # Nível máximo de compressão ZSTD
        "-co", "NUM_THREADS=ALL_CPUS",             # Utiliza todos os núcleos disponíveis, acelerando o processo
        "-co", "BIGTIFF=YES",                      # Permite a criação de arquivos GeoTIFF maiores que 4GB
        "-co", "SPARSE_OK=TRUE",                   # Permite a criação de blocos esparsos, economizando espaço em áreas nodata
        vrt_file,                                   # Arquivo VRT de entrada criado pelo gdalbuildvrt
        output_file                                 # Arquivo GeoTIFF de saída
    ]

    translate_command_float = [
        "gdal_translate",                          # Comando principal do GDAL para conversão de formato
        "-of", "GTiff",                            # Define o formato de saída como GeoTIFF
        "-ot", "Float32",                          # Tipo de dado de saída: Float de 32 bits (para dados contínuos)
        "-co", "TILED=YES",                        # Cria a saída em blocos (tiled), melhorando performance de leitura
        "-co", "COMPRESS=LZW",                    # Usa compressão ZSTD para melhor eficiência (alternativa a DEFLATE ou LZW)
        "-co", "PREDICTOR=3",                      # Ajusta o preditor para dados float (melhor compressão)
        "-co", "ZLEVEL=9",                         # Nível máximo de compressão ZSTD
        "-co", "NUM_THREADS=ALL_CPUS",             # Utiliza todos os núcleos disponíveis, acelerando o processo
        "-co", "BIGTIFF=YES",                      # Permite a criação de arquivos GeoTIFF maiores que 4GB
        "-co", "SPARSE_OK=TRUE",                   # Permite a criação de blocos esparsos, economizando espaço em áreas nodata
        vrt_file,                                   # Arquivo VRT de entrada criado pelo gdalbuildvrt
        output_file                                 # Arquivo GeoTIFF de saída
    ]
    ### ATENÇÃO!!!, AJUSTAR O DADO PARA CASO SEJA INT OU FLOAT
    # result = subprocess.run(translate_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    result = subprocess.run(translate_command_float, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode != 0:
        print("[ERRO] gdal_translate falhou:")
        print("STDERR:", result.stderr)
        print("STDOUT:", result.stdout)
        return False


    # Remover arquivos temporários
    if os.path.exists(vrt_file):
        os.remove(vrt_file)
    if os.path.exists('input_files.txt'):
        os.remove('input_files.txt')

    # Verificar se o arquivo de saída foi criado
    if not os.path.exists(output_file):
        print(f"[ERRO] O arquivo {output_file} não foi criado. Verifique os arquivos de entrada.")
        return False

    return True

def upload_file(bucket, source_file_name, destination_blob_name):
    """
    Faz upload de um arquivo para o bucket do Google Cloud Storage.

    Se o arquivo estiver salvo localmente (ex: /content/temp/), ele faz upload do arquivo local.
    Se o arquivo for remoto (ex: /vsigs/), a cópia é feita diretamente no GCS.

    Args:
        bucket: Cliente do GCS.
        source_file_name (str): Caminho do arquivo de origem.
        destination_blob_name (str): Caminho no bucket de destino.
    """
    blob = bucket.blob(destination_blob_name)

    if source_file_name.startswith("/content/"):  # O arquivo está salvo localmente no Colab
        try:
            blob.upload_from_filename(source_file_name)
            print(f"[UPLOAD ✅] {source_file_name} → {destination_blob_name}")
        except Exception as e:
            print(f"[ERRO ❌] Falha no upload de {source_file_name}: {e}")

    else:
        try:
            # Cópia entre objetos dentro do próprio GCS (se necessário)
            source_blob = bucket.blob(source_file_name)
            source_blob.copy_to(bucket.blob(destination_blob_name))
            print(f"[UPLOAD ✅] {source_file_name} → {destination_blob_name} (cópia GCS)")
        except Exception as e:
            print(f"[ERRO ❌] Falha na cópia no GCS: {e}")

import time

def process_mosaics(selected_bands, data_info, bucket_name, input_folder, output_folder):
    """
    Processa os arquivos selecionados:
    - Se for um mosaico, cria e salva o mosaico localmente antes de enviá-lo ao GCS.
    - Se não for um mosaico, apenas copia o arquivo para a subpasta correta.
    - No Google Colab, os arquivos temporários são excluídos após o upload.

    Args:
        selected_bands (dict): Bandas selecionadas para processamento.
        data_info (dict): Informações organizadas por tipo de dado e bandas.
        bucket_name (str): Nome do bucket de armazenamento.
        input_folder (str): Pasta de entrada no bucket.
        output_folder (str): Pasta de destino no bucket.
    """
    global stop_requested
    stop_requested = False
    start_time = time.time()
    total = sum(len(bands) for bands in selected_bands.values())
    done = 0

    print("\n" + "="*60)
    print(f"[🚀 INÍCIO] Processamento de {total} bandas selecionadas.")
    print("="*60)

    for data_type, bands in selected_bands.items():
        for band in bands:
            if stop_requested:
                print("[⚠️ INFO] Processamento interrompido pelo usuário.")
                return

            operation_start_time = time.time()
            remaining = total - done

            print("\n" + "-"*60)
            print(f"[🎵 PROCESSANDO] Banda: {data_type}-{band}")
            print(f"[📊 PROGRESSO] {done}/{total} processadas. Restam {remaining}.")
            print("-"*60)

            is_mosaic = data_info[data_type]['requires_mosaic']

            if is_mosaic:
                # Caminhos de entrada para mosaico
                group_files_paths = [f"/vsigs/{bucket_name}/{fp}" for fp in data_info[data_type]['bands'][band].get('files', [])]
                if not group_files_paths:
                    print(f"[❌ ERRO] Nenhum arquivo encontrado para mosaico {data_type}-{band}. Pulando...")
                    continue

                # Criar nome dos arquivos
                output_file_name = f"{data_type}/{data_type}-{band}.tif"
                output_file_local = f"/content/temp/{output_file_name}"  # Agora salva na pasta do Colab
                output_file_remote = f"{output_folder.rstrip('/')}/{output_file_name}"

                os.makedirs(os.path.dirname(output_file_local), exist_ok=True)

                # Se o mosaico já existe no bucket, pula
                if check_if_mosaic_exists(bucket_name, output_folder, data_type, band):
                    print(f"[✅ INFO] Mosaico já existe: {output_file_name}. Pulando...")
                    continue

                print(f"[🖼️ MOSAICO] Criando mosaico: {output_file_local}")
                success = mosaic_images(group_files_paths, output_file_local)

                # Verificar se o mosaico foi criado com sucesso
                if not success or not os.path.exists(output_file_local):
                    print(f"[❌ ERRO] Falha ao criar o mosaico para {output_file_local}. Pulando...")
                    continue

                # Upload do mosaico
                print(f"[📤 UPLOAD] Enviando mosaico {output_file_local} para {output_file_remote}")
                upload_file(bucket, output_file_local, output_file_remote)

                # Excluindo o arquivo temporário após o upload
                try:
                    os.remove(output_file_local)
                    print(f"[🗑️ LIMPEZA] Arquivo temporário excluído: {output_file_local}")
                except Exception as e:
                    print(f"[⚠️ AVISO] Não foi possível excluir {output_file_local}: {e}")

            else:
                # Copia de arquivo único (sem mosaico)
                source_file = f"/vsigs/{bucket_name}/{data_info[data_type]['bands'][band]['file']}"
                destination_path = f"{output_folder.rstrip('/')}/{data_type}/{os.path.basename(source_file)}"

                print(f"[📁 CÓPIA] Copiando arquivo único para: {destination_path}")
                upload_file(bucket, source_file, destination_path)

            done += 1
            elapsed_time = time.time() - start_time
            avg_time = elapsed_time / done if done > 0 else 0
            remaining_time = avg_time * (total - done) if avg_time > 0 else 0
            op_elapsed = time.time() - operation_start_time

            print(f"[📊 PROGRESSO] {done}/{total} bandas processadas.")
            print(f"[⏳ TEMPO] Decorrido: {format_time(elapsed_time)}")
            print(f"[⏳ TEMPO] Restante estimado: {format_time(remaining_time)}")
            print(f"[⏳ TEMPO] Tempo desta operação: {format_time(op_elapsed)}")

    print("\n" + "="*60)
    print(f"[✅ FINALIZADO] Processamento concluído para {total} bandas!")
    print(f"[⏳ TEMPO TOTAL] {format_time(time.time() - start_time)}")
    print("="*60)

def format_time(seconds):
    """
    Formata um tempo em segundos para um formato legível:
    - Se houver dias, exibe "Xd Yh Zm Ws".
    - Se não houver dias, exibe "Xh Ym Zs".
    - Se for menor que 1 hora, exibe apenas "Xm Ys".

    Args:
        seconds (float): Tempo em segundos.

    Returns:
        str: Tempo formatado corretamente.
    """
    days = int(seconds // 86400)  # 86400 segundos em um dia
    hours = int((seconds % 86400) // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)

    if days > 0:
        return f"{days}d {hours}h {minutes}m {secs}s"
    elif hours > 0:
        return f"{hours}h {minutes}m {secs}s"
    elif minutes > 0:
        return f"{minutes}m {secs}s"
    else:
        return f"{secs}s"  # Caso o tempo seja menor que 1 minuto



def load_data_info(bucket_name_out, input_folder, output_folder):
    """
    Carrega informações sobre os arquivos, identificando corretamente os tipos de dados e as bandas.
    Diferencia arquivos únicos dos que precisam ser mosaicados.

    - O tipo de dado é sempre o primeiro elemento ao fazer split('-').
    - A banda é sempre o segundo elemento ao fazer split('-'), mantendo _YYYY ou _YYYY_YYYY.
    - Se o arquivo for um mosaico, remover os últimos 21 caracteres antes de pegar o nome da banda.
    - Se o arquivo não for um mosaico, apenas remover a extensão .tif.

    Args:
        bucket_name_out (str): Nome do bucket de saída.
        input_folder (str): Pasta onde os arquivos estão armazenados.
        output_folder (str): Pasta onde os mosaicos serão armazenados.

    Returns:
        dict: Estrutura contendo tipos de dados e bandas associadas.
    """
    files = list_files_in_folder(bucket_name_out, input_folder)
    total = len(files)
    processed = 0

    print(f"[INFO] Iniciando carregamento de informações dos arquivos.")
    print(f"[INFO] Total de arquivos: {total}")

    data_info = {}

    for f in files:
        processed += 1
        remaining = total - processed

        print(f"[PROGRESSO] Processando arquivo {processed}/{total}... Restam {remaining} arquivos.")

        filename = os.path.basename(f)

        # Garantir que o arquivo é um TIFF
        if not filename.endswith('.tif'):
            print(f"Aviso: O arquivo {filename} não é um TIFF válido. Ignorando...")
            continue

        # Remover extensão
        base_name = filename[:-4]

        # Identificar se o arquivo precisa ser mosaicado pelo padrão numérico no final (21 caracteres)
        is_mosaic = re.search(r"\d{10}-\d{10}$", base_name)

        if is_mosaic:
            clean_name = base_name[:-21]  # Remover os 21 caracteres finais
        else:
            clean_name = base_name  # Arquivo único

        # O tipo de dado é sempre o primeiro elemento do split('-')
        parts = clean_name.split('-')
        data_type = parts[0]  # Primeiro elemento é o tipo de dado

        # O nome da banda é sempre o segundo elemento do split('-')
        if len(parts) > 1:
            band = parts[1]
        else:
            print(f"Aviso: O nome do arquivo {filename} não tem estrutura esperada para a banda. Ignorando...")
            continue

        # Verificar se o mosaico já existe
        try:
            mosaic_exists = check_if_mosaic_exists(bucket_name_out, output_folder, data_type, band)
        except Exception as e:
            print(f"Erro ao verificar mosaico para {filename}: {e}")
            mosaic_exists = False

        # Organizar os dados no dicionário
        if data_type not in data_info:
            data_info[data_type] = {'bands': {}, 'requires_mosaic': bool(is_mosaic)}

        if band not in data_info[data_type]['bands']:
            data_info[data_type]['bands'][band] = {'files': [], 'mosaic_exists': mosaic_exists}

        # Se for mosaico, adicionar na lista de arquivos para mosaico
        if is_mosaic:
            data_info[data_type]['bands'][band]['files'].append(f)
        else:
            # Se não for mosaico, apenas armazenar como um único arquivo
            data_info[data_type]['bands'][band]['file'] = f

    print("[INFO] Carregamento concluído.")

    # Debugging para verificar se os dados estão corretos antes de enviar para a interface
    print("\n🔍 Tipos de dados encontrados:", list(data_info.keys()))
    for dt, info in data_info.items():
        print(f"\n🗂️ Tipo de dado: {dt}")
        print("🎵 Bandas:", list(info["bands"].keys()))

    return data_info


In [4]:
# Carregar as informações
data_info = load_data_info(bucket_name, input_folder, output_folder)


[INFO] Iniciando carregamento de informações dos arquivos.
[INFO] Total de arquivos: 360
[PROGRESSO] Processando arquivo 1/360... Restam 359 arquivos.
[PROGRESSO] Processando arquivo 2/360... Restam 358 arquivos.
[PROGRESSO] Processando arquivo 3/360... Restam 357 arquivos.
[PROGRESSO] Processando arquivo 4/360... Restam 356 arquivos.
[PROGRESSO] Processando arquivo 5/360... Restam 355 arquivos.
[PROGRESSO] Processando arquivo 6/360... Restam 354 arquivos.
[PROGRESSO] Processando arquivo 7/360... Restam 353 arquivos.
[PROGRESSO] Processando arquivo 8/360... Restam 352 arquivos.
[PROGRESSO] Processando arquivo 9/360... Restam 351 arquivos.
[PROGRESSO] Processando arquivo 10/360... Restam 350 arquivos.
[PROGRESSO] Processando arquivo 11/360... Restam 349 arquivos.
[PROGRESSO] Processando arquivo 12/360... Restam 348 arquivos.
[PROGRESSO] Processando arquivo 13/360... Restam 347 arquivos.
[PROGRESSO] Processando arquivo 14/360... Restam 346 arquivos.
[PROGRESSO] Processando arquivo 15/360

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Calcular métricas:
files_count = sum(len(b['files']) for d in data_info.values() for b in d['bands'].values())
folder_name = input_folder.rstrip('/').split('/')[-1]  # nome da pasta temp observada
data_types_count = len(data_info)
bands_per_data_type = {dt: len(info['bands']) for dt, info in data_info.items()}
mosaic_count = sum(1 for d in data_info.values() for b in d['bands'].values() if b['mosaic_exists'])

# Label com métricas
met_label = widgets.HTML(f"""
<p><b>Pasta observada:</b> {folder_name}<br>
<b>Número total de arquivos:</b> {files_count}<br>
<b>Número de tipos de dados:</b> {data_types_count}<br>
<b>Bandas por tipo de dado:</b> {', '.join([f"{dt}: {c}" for dt,c in bands_per_data_type.items()])}<br>
<b>Bandas já mosaicadas:</b> {mosaic_count}
</p>
""")

# Interface de seleção
data_type_checkboxes = {}  # Guardar a estrutura de checkboxes por tipo de dado
accordion_items = []
accordion_titles = []

def toggle_data_type(dt):
    """Liga/desliga todas as bandas de um mesmo tipo de dado (dt) não desabilitadas."""
    dt_checkboxes = data_type_checkboxes[dt]
    enabled_checkboxes = [cb.value for band, cb in dt_checkboxes.items() if not cb.disabled]
    # Se todos ligados, desliga; se algum desligado, liga todos
    if enabled_checkboxes and all(enabled_checkboxes):
        new_value = False
    else:
        new_value = True

    for band, cb in dt_checkboxes.items():
        if not cb.disabled:
            cb.value = new_value

for dt, info in data_info.items():
    data_type_checkboxes[dt] = {}
    # Cria os checkboxes de banda
    vbox_bandas = []
    for band, band_info in info['bands'].items():
        cb = widgets.Checkbox(
            value=(not band_info['mosaic_exists']),
            description=(f"{band}{' ⚠️' if band_info['mosaic_exists'] else ''}"),
            disabled=band_info['mosaic_exists']
        )
        data_type_checkboxes[dt][band] = cb
        vbox_bandas.append(cb)

    # Botão para ligar/desligar apenas as bandas deste tipo de dado
    dt_toggle_button = widgets.Button(
        description="Ligar/Desligar Esse Tipo",
        button_style='info',
        layout=widgets.Layout(width='auto')
    )

    # Função de callback para o botão do tipo de dado
    def make_dt_callback(data_type):
        def callback(b):
            toggle_data_type(data_type)
        return callback

    dt_toggle_button.on_click(make_dt_callback(dt))

    # Agrupa o botão e as bandas em um único VBox
    dt_box = widgets.VBox([dt_toggle_button] + vbox_bandas)

    accordion_items.append(dt_box)
    accordion_titles.append(dt)

accordion = widgets.Accordion(children=accordion_items)
for i, title in enumerate(accordion_titles):
    accordion.set_title(i, title)

instructions_label = widgets.HTML("""
<p>Selecione as bandas que deseja mosaicar. As bandas já mosaicadas são marcadas com ⚠️ e estão desabilitadas.<br>
Use o botão global para ligar/desligar todas as bandas, ou o botão específico dentro de cada tipo de dado para ligar/desligar somente as bandas daquele tipo.<br>
Após ajustar as seleções, utilize outra célula para coletar essas seleções e iniciar o mosaico manualmente.</p>
""")

toggle_all_button = widgets.Button(description="Ligar/Desligar Todas", button_style='info')
output_area = widgets.Output()

def on_toggle_all_clicked(b):
    # Se todas habilitadas estão ligadas, desliga tudo, senão liga tudo
    all_enabled = []
    for dt, bands_dict in data_type_checkboxes.items():
        for band, cb in bands_dict.items():
            if not cb.disabled:
                all_enabled.append(cb.value)
    if all_enabled and all(all_enabled):
        new_value = False
    else:
        new_value = True
    for dt, bands_dict in data_type_checkboxes.items():
        for band, cb in bands_dict.items():
            if not cb.disabled:
                cb.value = new_value

toggle_all_button.on_click(on_toggle_all_clicked)

display(met_label, instructions_label, toggle_all_button, accordion, output_area)


HTML(value='\n<p><b>Pasta observada:</b> temp<br>\n<b>Número total de arquivos:</b> 360<br>\n<b>Número de tipo…

HTML(value='\n<p>Selecione as bandas que deseja mosaicar. As bandas já mosaicadas são marcadas com ⚠️ e estão …

Button(button_style='info', description='Ligar/Desligar Todas', style=ButtonStyle())

Accordion(children=(VBox(children=(Button(button_style='info', description='Ligar/Desligar Esse Tipo', layout=…

Output()

In [ ]:
selected_bands = {}
for dt, bands_dict in data_type_checkboxes.items():
    selected = [band for band, cb in bands_dict.items() if cb.value and (not cb.disabled)]
    if selected:
        selected_bands[dt] = selected

if not selected_bands:
    print("Nenhuma banda selecionada.")
else:
    print("Bandas selecionadas:", selected_bands)
    # Aqui você pode chamar a função de processamento manualmente, remova o comentario para
    # disparar o mosaico para os checks que estiverem ligados na celula anterior:
    process_mosaics(selected_bands, data_info, bucket_name, input_folder, output_folder)


Bandas selecionadas: {'nativeMask': ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_2023', 'classification_2024']}

[🚀 INÍCIO] Processamento de 40 bandas sel